In [208]:
%%writefile Housing.py
import pandas as pd
import numpy as np
import matplotlib.style as style
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import re
from datetime import datetime
from collections import defaultdict
import itertools
from random import choices
from collections import Counter
import math
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
from math import sqrt,ceil
from sklearn.metrics import mean_squared_error
style.use('fivethirtyeight')
pd.options.display.max_columns = 999
pd.options.display.float_format = '{:,.3f}'.format
#%matplotlib inline 
from sklearn import preprocessing, linear_model
from sklearn.linear_model import LassoCV,Lasso, RidgeCV, Ridge
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import normalize
import csv
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import sys
import pickle
path = 'd:/ssjiun/Project Folder/data/'

class HousingModel():
    def __init__(self,df,Type):
        self.df = df
        self.Type = Type
        if self.Type not in ['Sale','Rent']:
            print('Please key in Type=Sale or Type=Rent')
        else:
            self.variableY = {'Sale':'price','Rent':'Price_persqft'}[self.Type]
        if self.variableY in self.df.columns:
            print('Log Transformation of ' + self.variableY + ' is completed')
        else:
            print('Variable Y is not found in data frame')
        self.logprice_name = 'logprice'
        self.df[self.logprice_name] = np.log(self.df[self.variableY])
        
    def processData(self,features_list):
        print('Converting column types....categorical -> dummy')
        if len(set(features_list) & set(self.df.columns)) != len(features_list):
            print('one or more columns in list of features is/are not found in data frame')
            sys.exit()
        catColumn_list = [i for i in features_list if self.df[i].dtype == 'object']
        self.df_unique = self.df.drop_duplicates()
        if catColumn_list:
            df_numerical =  self.df_unique[list(set(features_list).difference(set(catColumn_list)))]
            df_numericalRaw = self.df[list(set(features_list).difference(set(catColumn_list)))]
            self.df_features = pd.concat([pd.get_dummies(self.df_unique[catColumn_list]),
                                        df_numerical],join_axes = [df_numerical.index], axis = 1).copy()
            self.df_featuresRaw = pd.concat([pd.get_dummies(self.df[catColumn_list]),
                                        df_numericalRaw],join_axes = [df_numericalRaw.index], axis = 1).copy()
        else:
            self.df_features = self.df_unique.copy()
            self.df_featuresRaw = self.df.copy()
        self.train_X, self.test_X, self.train_Y, self.test_Y = train_test_split(self.df_features, 
                                                                                self.df_unique[self.logprice_name],
                                                                               test_size = 0.25, random_state = 42)
        print('Finished data splitting into test and training set')
                
    def trainModel(self,features_list,modelType,label = ''):
        self.modelType = modelType            
        if self.modelType not in ['LM','Lasso','Ridge','RF']:
            print('please key in LM, Lasso, Ridge or RF for modeltype parameter')
            sys.exit()     
        print('Perform 5 fold cross-validation to find best hyperparameter if Lasso/Ridge')
        alpha_list = np.logspace(-4, 4, 10)
        if self.modelType == 'Lasso':
            self.model = LassoCV(alphas = alpha_list, normalize = False, cv = 5).fit(self.train_X,self.train_Y)
            self.trainedModel = Lasso(alpha = self.model.alpha_).fit(self.train_X, self.train_Y) 
        elif self.modelType == 'Ridge':
            self.model = RidgeCV(alphas = alpha_list, normalize = False, cv = 5).fit(self.train_X,self.train_Y)
            self.trainedModel = Ridge(alpha = self.model.alpha_).fit(self.train_X, self.train_Y) 
        elif self.modelType == 'RF':
            self.model = RandomForestRegressor(n_estimators = 100, random_state = 42)
            self.trainedModel = self.model.fit(self.train_X, self.train_Y)
        elif self.modelType == 'LM':
            self.model = linear_model.LinearRegression()
            self.trainedModel = self.model.fit(self.train_X, self.train_Y)
        filename = self.modelType + label + '_' + datetime.now().strftime('%d-%m-%Y') + '.sav'
        pickle.dump(self.trainedModel, open(path + 'model/' + filename, 'wb'))
        print('Model - {} is trained and saved'.format(filename))
        return self.trainedModel
        
    def rankFeatures(self,modelType = None,filepath = None):
        if modelType:
            self.modelType = modelType
        if filepath:
            self.trainedModel = pickle.load(open(filepath,'rb'))
        if self.modelType in ['Lasso','Ridge']:
            featuresScore_list = sorted([(i,j) for i,j in zip(self.df_features.columns,
                                self.trainedModel.coef_)],key = lambda x:x[1],reverse = True)
        elif self.modelType == 'RF':
            featuresScore_list = sorted([(i,j) for i,j in zip(self.df_features.columns,
                            self.trainedModel.feature_importances_)],key = lambda x:x[1],reverse = True)
        elif self.modelType == 'LM':
            featuresScore_list = sorted([(i,j) for i,j in zip(self.df_features.columns,
                            self.trainedModel.coef_)],key = lambda x:x[1],reverse = True)
        featuresResult_df = pd.DataFrame(featuresScore_list) 
        featureResult_df.columns = ['Feature','FeatureCoef']
        featureResult_df.to_excel(path + 'features/FeaturesResult_%s%s.xlsx'%(self.Type,self.modelType),index = False)
        print('Features Result is saved in {}'.format(path + 'features/FeaturesResult_%s%s.xlsx'%(self.Type,self.modelType)))
        return result_df
    
    def prediction(self, modelType = None, filepath = None):
        MeanPrice_dict = {}
        if filepath:
            self.trainedModel = pickle.load(open(filepath, 'rb'))
        self.df['estimatedPrice'] = self.trainedModel.predict(self.df_featuresRaw)
        for yearqr in self.df['year-quarter'].unique():
            MeanPrice_dict[str(yearqr)] = np.mean(self.df[self.df['year-quarter'] == yearqr]['estimatedPrice'])
        
        #computation of price index
        return {i:(np.exp(j)/np.exp(MeanPrice_dict['2013-1']))*100 for i,j in MeanPrice_dict.items()}
        
    def evaluateModel(self, modelType = None, filepath = None):
        modelResult_dict = {}
        if modelType:
            self.modelType = modelType
        if filepath:
            self.trainedModel = pickle.load(open(filepath,'rb'))
        if self.modelType in ['Lasso','Ridge']:
            train_rmse = np.mean(np.sqrt(-cross_val_score(self.trainedModel, self.train_X, self.train_Y,
                                                  scoring = 'neg_mean_squared_error',cv = 5)))
            test_rmse = sqrt(mean_squared_error(self.test_Y,self.trainedModel.predict(self.test_X)))
        elif self.modelType in ['RF','LM']:
            train_rmse = sqrt(mean_squared_error(self.train_Y,self.trainedModel.predict(self.train_X)))
            test_rmse = sqrt(mean_squared_error(self.test_Y,self.trainedModel.predict(self.test_X)))
        modelResult_dict['Train_RMSE'] = train_rmse
        modelResult_dict['Test_RMSE'] = test_rmse
        return modelResult_dict

Writing Housing.py


# Rental

In [ ]:
#import columns
cols_list = []
with open(path + 'excel/externalData_cols.csv') as newfile:
    readfile = csv.reader(newfile)
    for row in readfile:
        cols_list.append(row)
exchangeRate_list = cols_list[0]
macro_list = cols_list[1]

# Random Forest

In [60]:
df_merged.to_excel(path + 'processed/rental_exchangeRate.xlsx')

KeyboardInterrupt: 

In [179]:
df_merged

Type                      Location  \
0       Condominium                           NaN   
1       Condominium                           NaN   
2       Condominium                           NaN   
3       Condominium                           NaN   
4       Condominium                           NaN   
5       Condominium                           NaN   
6       Condominium                           NaN   
7       Condominium                           NaN   
8       Condominium                           NaN   
9       Condominium                           NaN   
10      Condominium                           NaN   
11      Condominium                           NaN   
12      Condominium                           NaN   
13      Condominium        Tanjung Tokong, Penang   
14      Condominium                           NaN   
15      Condominium      Titiwangsa, Kuala Lumpur   
16            House      Kuala Selangor, Selangor   
17      Condominium  Ara Damansara, Petaling Jaya   
18      Condominium                           NaN   
19      Condominium                           NaN   
20      Condominium                           NaN   
21      Condominium                           NaN   
22      Condominium                           NaN   
23      Condominium                           NaN   
24      Condominium                           NaN   
25      Condominium         Keramat, Kuala Lumpur   
26      Condominium                           NaN   
27      Condominium                           NaN   
28            House           Putrajaya, Selangor   
29      Condominium   Kenny Heights, Kuala Lumpur   
...             ...                           ...   
486334  Condominium                           NaN   
486335  Condominium                           NaN   
486336  Condominium                           NaN   
486337  Condominium                           NaN   
486338  Condominium                           NaN   
486339  Condominium                           NaN   
486340  Condominium                           NaN   
486341  Condominium                           NaN   
486342  Condominium                           NaN   
486343  Condominium                           NaN   
486344  Condominium                           NaN   
486345  Condominium         Hampshire Place, KLCC   
486346  Condominium                           NaN   
486347  Condominium                           NaN   
486348  Condominium         Wisma Nusantara, KLCC   
486349  Condominium                           NaN   
486350  Condominium                           NaN   
486351  Condominium         Wisma Nusantara, KLCC   
486352  Condominium                           NaN   
486353  Condominium                           NaN   
486354  Condominium         Menara Hap Seng, KLCC   
486355  Condominium                           NaN   
486356  Condominium                           NaN   
486357  Condominium         Wisma Nusantara, KLCC   
486358  Condominium         Wisma Nusantara, KLCC   
486359  Condominium         Wisma Nusantara, KLCC   
486360  Condominium         Wisma Nusantara, KLCC   
486361  Condominium                           NaN   
486362  Condominium                           NaN   
486363  Condominium                           NaN   

                                 Property     price  size       layout  \
0                    Pulau Pangkor, Perak 4,300.000  1700    3-Bedroom   
1                       Batu Pahat, Johor 1,300.000  1150    3-Bedroom   
2                Taman Desa, Kuala Lumpur 1,950.000  1345    4-Bedroom   
3                  Sungai Buloh, Selangor 1,400.000  1205    3-Bedroom   
4                Taman Desa, Kuala Lumpur 2,200.000  1115    3-Bedroom   
5                    Tanjung Malim, Perak 1,200.000   900    3-Bedroom   
6                  Sungai Buloh, Selangor 1,600.000  1001    2-Bedroom   
7                  Sungai Buloh, Selangor 1,600.000   936    3-Bedroom   
8                  Sungai Buloh, Selangor 1,600.000  1023    3

In [178]:
#Preprocessing
columns_list = ['furnishing_bool','num_bedroom','bath','carpark','district_master',
            'Type_le','ClosetoLRT','ClosetoKTM','ClosetoMRT','ClosetoMonorail','year_str','year-quarter']\
                + exchangeRate_list

df_merged2 = df_merged.sort_values('date')[columns_list + ['Price_persqft']].copy()

#preprocessing
#ffill and bfill missing values
for i in exchangeRate_list:
    df_merged2[i] = df_merged2[i].fillna(method = 'ffill').fillna(method = 'bfill')
    print(sum(pd.isnull(df_merged2[i])))

KeyError: 'THB100'

In [197]:
rental = HousingModel(df_merged, 'Rent')

Log Transformation of Price_persqft is completed


In [209]:
columns_list = ['furnishing','num_bedroom','bath','carpark','district_master',
            'Type','year_str'] 
rental.processData(columns_list)

Converting column types....categorical -> dummy
Finished data splitting into test and training set


In [193]:
rental.trainModel(columns_list,'Lasso','Internal')

Perform 5 fold cross-validation to find best hyperparameter if Lasso/Ridge
Model - LassoInternal_04-07-2018.sav is trained and saved


Lasso(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [211]:
rental.evaluateModel('Lasso', path + 'model/LassoInternal_04-07-2018.sav')

{'Test_RMSE': 0.2430835514517637, 'Train_RMSE': 0.24195944982676226}

In [194]:
rental.trainModel(columns_list,'Ridge','Internal')

Perform 5 fold cross-validation to find best hyperparameter if Lasso/Ridge
Model - RidgeInternal_04-07-2018.sav is trained and saved


Ridge(alpha=0.046415888336127774, copy_X=True, fit_intercept=True,
   max_iter=None, normalize=False, random_state=None, solver='auto',
   tol=0.001)

In [212]:
rental.evaluateModel('Ridge', path + 'model/RidgeInternal_04-07-2018.sav')

{'Test_RMSE': 0.24265044887203618, 'Train_RMSE': 0.24144168197764831}

In [199]:
rental.trainModel(columns_list,'LM','Internal')

Perform 5 fold cross-validation to find best hyperparameter if Lasso/Ridge
Model - LMInternal_04-07-2018.sav is trained and saved


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [213]:
rental.evaluateModel('LM', path + 'model/LMInternal_04-07-2018.sav')

{'Test_RMSE': 1859807122.0138574, 'Train_RMSE': 0.24207839138770246}

In [201]:
#with internal factors only
columns_list = ['furnishing_bool','num_bedroom','bath','carpark','district_master',
            'Type_le','year_str'] 
rental.processData(columns_list)
rental.trainModel(columns_list,'RF','Internal')

Converting column types....categorical -> dummy
Finished data splitting into test and training set
Perform 5 fold cross-validation to find best hyperparameter if Lasso/Ridge
Model - RFInternal_04-07-2018.sav is trained and saved


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [214]:
#with internal factors only
columns_list = ['furnishing_bool','num_bedroom','bath','carpark','district_master',
            'Type_le','year_str'] 
rental.processData(columns_list)
rental.evaluateModel('RF', path + 'model/RFInternal_04-07-2018.sav ')

Converting column types....categorical -> dummy
Finished data splitting into test and training set


{'Test_RMSE': 0.2362753418861446, 'Train_RMSE': 0.23235818059346525}

In [215]:
columns_list = ['furnishing','num_bedroom','bath','carpark','district_master',
            'Type','year_str','ClosetoLRT','ClosetoKTM','ClosetoMRT','ClosetoMonorail'] 
rental.processData(columns_list)

Converting column types....categorical -> dummy
Finished data splitting into test and training set


In [203]:
rental.trainModel(columns_list,'Lasso','InternalExt')

Perform 5 fold cross-validation to find best hyperparameter if Lasso/Ridge
Model - LassoInternalExt_04-07-2018.sav is trained and saved


Lasso(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [216]:
rental.evaluateModel('Lasso', path + 'model/LassoInternalExt_04-07-2018.sav')

{'Test_RMSE': 0.2428826403246105, 'Train_RMSE': 0.24176494428835965}

In [204]:
rental.trainModel(columns_list,'Ridge','InternalExt')

Perform 5 fold cross-validation to find best hyperparameter if Lasso/Ridge
Model - RidgeInternalExt_04-07-2018.sav is trained and saved


Ridge(alpha=0.046415888336127774, copy_X=True, fit_intercept=True,
   max_iter=None, normalize=False, random_state=None, solver='auto',
   tol=0.001)

In [217]:
rental.evaluateModel('Ridge', path + 'model/RidgeInternalExt_04-07-2018.sav')

{'Test_RMSE': 0.24244646630107913, 'Train_RMSE': 0.24124377172199632}

In [205]:
rental.trainModel(columns_list,'LM','InternalExt')

Perform 5 fold cross-validation to find best hyperparameter if Lasso/Ridge
Model - LMInternalExt_04-07-2018.sav is trained and saved


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [218]:
rental.evaluateModel('LM', path + 'model/LMInternalExt_04-07-2018.sav')

{'Test_RMSE': 1956623728.2001169, 'Train_RMSE': 0.24123088454995784}

In [206]:
columns_list = ['furnishing_bool','num_bedroom','bath','carpark','district_master',
            'Type_le','year_str','ClosetoLRT','ClosetoKTM','ClosetoMRT','ClosetoMonorail'] 
rental.processData(columns_list)
rental.trainModel(columns_list,'RF','InternalExt')

Converting column types....categorical -> dummy
Finished data splitting into test and training set
Perform 5 fold cross-validation to find best hyperparameter if Lasso/Ridge
Model - RFInternalExt_04-07-2018.sav is trained and saved


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [219]:
columns_list = ['furnishing_bool','num_bedroom','bath','carpark','district_master',
            'Type_le','year_str','ClosetoLRT','ClosetoKTM','ClosetoMRT','ClosetoMonorail'] 
rental.processData(columns_list)
rental.evaluateModel('RF', path + 'model/RFInternalExt_04-07-2018.sav')

Converting column types....categorical -> dummy
Finished data splitting into test and training set


{'Test_RMSE': 0.22921768864409275, 'Train_RMSE': 0.22427275605987818}

In [221]:
result_df = pd.DataFrame({'Internal Training RMSE':[0.24195944982676226,0.24144168197764831,0.24207839138770246,0.23235818059346525],
              'Internl Test RMSE':[0.2430835514517637,0.24265044887203618,1859807122.0138574,0.2362753418861446],
 'Internal + External Training RMSE': [0.24176494428835965,0.24124377172199632,0.24123088454995784,0.22427275605987818], 
'Internal + External Test RMSE': [0.2428826403246105,0.24244646630107913,1956623728.2001169,0.22921768864409275]},
            index = ['Lasso','Ridge','LM','RF'])

In [222]:
result_df.to_excel(path + 'excel/modelResult.xlsx')

In [160]:
#from py file inside predictive folder
from predictive.Housing import HousingModel as HM

rental = HM(df_merged,'Rent')

columns_list = ['furnishing_bool','num_bedroom','bath','carpark','district_master',
            'Type_le','ClosetoLRT','ClosetoKTM','ClosetoMRT','ClosetoMonorail','year_str'] 
rental.processData(columns_list)

rental.rankFeatures(modelType = 'RF',filepath = path + 'model/RF.sav')

Log Transformation of Price_persqft is completed
